# [Task](https://docs.google.com/document/d/1cYDE1QOHwHYu1QkOTtvsVw0NZ0P1piGt9Rna62iBt18/edit?tab=t.0)

In [74]:
import sympy as sp

Due to simplicity of system we can derive the linear jacobians of CoM (center of mass) by geometric approach

In [75]:
D_1 = sp.Symbol("d_1")
M_1, M_2 = sp.symbols("m_1,m_2")
I_1, I_2 = sp.Symbol("I_1") * sp.eye(3), sp.Symbol("I_2") * sp.eye(3)
TIME = sp.Symbol("t")
QS = sp.Function("q_1")(TIME), sp.Function("q_2")(TIME)
G = sp.Symbol("g")


def skew_sym_sim(v: sp.Matrix) -> sp.Matrix:
    if len(v.shape) != 1:
        v = sp.Matrix([v[0, 0], v[1, 0], v[2, 0]])
    return sp.Matrix([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])


def rot_z_sim(theta: sp.Symbol) -> sp.Matrix:
    return sp.Matrix(
        [
            [sp.cos(theta), -sp.sin(theta), 0],
            [sp.sin(theta), sp.cos(theta), 0],
            [0, 0, 1],
        ]
    )


def jac_vel_1_cm_sim(q_1: sp.Symbol) -> sp.Matrix:
    res = sp.zeros(3, 2)
    r_cm_1 = rot_z_sim(q_1) @ sp.Matrix([[D_1 / 2], [0], [0]])
    res[:, 0] = skew_sym_sim(-r_cm_1)[:, 2]
    return sp.Matrix(res)


def jac_vel_2_cm_sim(q_1: sp.Symbol, q_2: sp.Symbol) -> sp.Matrix:
    res = sp.zeros(3, 2)
    # CoM 2 position
    r_j_2 = rot_z_sim(q_1) @ sp.Matrix([[D_1], [q_2], [0]])
    res[:, 0] = skew_sym_sim(-r_j_2)[:, 2]
    # add relative velocity of CoM 2
    res[:, 1] = rot_z_sim(q_1)[:, 1]
    return sp.Matrix(res)


jac_vel_2_cm_sim(*QS)

Matrix([
[-d_1*sin(q_1(t)) - q_2(t)*cos(q_1(t)), -sin(q_1(t))],
[ d_1*cos(q_1(t)) - q_2(t)*sin(q_1(t)),  cos(q_1(t))],
[                                    0,            0]])

Angular jacobians should be obtained in the body reference frame. The second joint is translational, so the angular velocity would be the same

In [76]:
def jac_ang_1_sim() -> sp.Matrix:
    return sp.Matrix(
        [
            [0, 0],
            [0, 0],
            [1, 0],
        ]
    )


def jac_ang_2_sim() -> sp.Matrix:
    return jac_ang_1_sim()


jac_ang_2_sim()

Matrix([
[0, 0],
[0, 0],
[1, 0]])

Now we can obtain directly terms of canonical manipulator equation, details can be found [here](http://www.cpdee.ufmg.br/~torres/wp-content/uploads/2018/02/Robot_Dynamics_part_1.pdf)

In [77]:
def get_mass_matrix(q_1: sp.Symbol, q_2: sp.Symbol) -> sp.Matrix:
    mass_matrix = sp.zeros(2, 2)
    j_v_1_cm = jac_vel_1_cm_sim(q_1)
    j_v_2_cm = jac_vel_2_cm_sim(q_1, q_2)
    j_a_1 = jac_ang_1_sim()
    j_a_2 = jac_ang_2_sim()
    mass_matrix += (M_1 * j_v_1_cm.T @ j_v_1_cm + j_a_1.T @ I_1 @ j_a_1)
    mass_matrix += (M_2 * j_v_2_cm.T @ j_v_2_cm + j_a_2.T @ I_2 @ j_a_2)
    return sp.simplify(mass_matrix)

MASS_MATRIX = get_mass_matrix(*QS)
MASS_MATRIX

Matrix([
[I_1 + I_2 + d_1**2*m_1/4 + d_1**2*m_2 + m_2*q_2(t)**2, d_1*m_2],
[                                              d_1*m_2,     m_2]])

In [78]:
# Symmetric
MASS_MATRIX - MASS_MATRIX.T

Matrix([
[0, 0],
[0, 0]])

In [79]:
def christoffel(i: int, j: int, k: int) -> sp.Symbol:
    return sp.simplify(
        (
            MASS_MATRIX[k, j].diff(QS[i])
            + MASS_MATRIX[k, i].diff(QS[j])
            - MASS_MATRIX[i, j].diff(QS[k])
        )
        / 2
    )


def coriolis() -> sp.Matrix:
    res = sp.zeros(2, 2)
    for k in range(2):
        for j in range(2):
            for i in range(2):
                res[k, j] += christoffel(i, j, k) * QS[i].diff(TIME)
    return sp.simplify(res)

CORIOLIS_MAT = coriolis()
CORIOLIS_MAT

Matrix([
[ m_2*q_2(t)*Derivative(q_2(t), t), m_2*q_2(t)*Derivative(q_1(t), t)],
[-m_2*q_2(t)*Derivative(q_1(t), t),                                0]])

In [80]:
# Skew symmetric
(MASS_MATRIX.diff(TIME) - 2 * CORIOLIS_MAT) + (MASS_MATRIX.diff(TIME) - 2 * CORIOLIS_MAT).T

Matrix([
[0, 0],
[0, 0]])

In [81]:
def potential_energy(q_1: sp.Symbol, q_2: sp.Symbol):
    r_cm_1 = rot_z_sim(q_1) @ sp.Matrix([[D_1 / 2], [0], [0]])
    p_1 = -r_cm_1[0, 0] * M_1 * G
    r_cm_2 = rot_z_sim(q_1) @ sp.Matrix([[D_1], [q_2], [0]])
    p_2 = -r_cm_2[0, 0] * M_2 * G
    return sp.simplify(p_1 + p_2)


def gravity_force(q_1: sp.Symbol, q_2: sp.Symbol):
    p_e = potential_energy(q_1, q_2)
    p_e = sp.Matrix([p_e])
    return sp.simplify(p_e.jacobian(sp.Matrix([q_1, q_2])))


gravity_force(*QS)

Matrix([[g*(d_1*m_1*sin(q_1(t)) + 2*m_2*(d_1*sin(q_1(t)) + q_2(t)*cos(q_1(t))))/2, g*m_2*sin(q_1(t))]])